In [31]:
import pandas as pd
import datetime
import numpy as np
import random

In [32]:
def load_dataframes():
    
    global days_data, monday, tuesday, wednesday, thursday, friday
    monday = pd.read_csv('./data/monday.csv', sep=';')
    tuesday = pd.read_csv('./data/tuesday.csv', sep=';')
    wednesday = pd.read_csv('./data/wednesday.csv', sep=';')
    thursday = pd.read_csv('./data/thursday.csv', sep=';')
    friday = pd.read_csv('./data/friday.csv', sep=';')
    days_data = [monday, tuesday, wednesday, thursday, friday]
    print('data was loaded.')
    #print(monday)
    return days_data, monday, tuesday, wednesday, thursday, friday
#load_data()


def new_columns():
    
    '''add new column day'''
    monday['day'] = 'monday'
    tuesday['day'] = 'tuesday'
    wednesday['day'] = 'wednesday'
    thursday['day'] = 'thursday'
    friday['day'] = 'friday'
    
    '''set datetime to index for every day'''
    for day in days_data:
        day.timestamp = pd.to_datetime(day.timestamp)
        day.set_index('timestamp', inplace=True)
        
        ''' add new column with day and customer_no'''
        list_of_IDs = []
        for unit in day['customer_no']:
            #print(unit)
            weekday = day['day'][0][:3]
            #print( unit, weekday)
            list_of_IDs.append(str(unit) +'-'+ weekday)
        day['day_id'] = list_of_IDs     
    
    print('new columns were added.')
    #print(monday)
    return days_data, monday, tuesday, wednesday, thursday, friday
#new_columns()


def get_non_checkout(df):
    non_checkout_customers = df['customer_no'].max()
    checkout_customers = []
    for c_id in range(non_checkout_customers):
        if not 'checkout' in df[df['customer_no'] == c_id+1]['location'].values:
            checkout_customers.append(c_id+1)
    print('a list of non checkouts was built')
    return checkout_customers

def add_checkout(df, customer_nos, date, day):
    df_fill = pd.DataFrame()
    for p_id in customer_nos:
        #print(p_id)
        trial_id = df.iloc[p_id]['day_id']
        trial_day = df['day'][0][:3]
        #print(p_id, trial_id, trial_day)
        trial_together = (str(p_id) + '-' + trial_day)
        #print(trial_together)
        df_tmp = pd.DataFrame(data=[[p_id, 'checkout', day, trial_together]],
                              index=[pd.to_datetime(f'2019-09-{date} 21:59:00')],
                              columns=['customer_no', 'location', 'day', 'day_id'])
        df = pd.concat([df, df_tmp])
    print(f'the checkout was added to the data-frame/customers for {day}.')
    
    return df

def per_minute(df):
    global total
    #total.reset_index()
    #total = total.groupby(['day_id']).resample('1Min').ffill()
    total = total.groupby(['day_id'])['location'].resample('1Min').ffill()

    #total = total.groupby(total['day_id'])['location'].resample('1Min').ffill()
    return total


if __name__=='__main__':
    print('*** S T A R T ***')
    load_dataframes()
    
    print('\n *** N E W   C O L U M N S ***')
    new_columns()
    
    print('\n *** A D D   N O   C H E C K O U T ***')
    customers_to_checkout = []
    for day in days_data:
        customers_to_checkout.append(get_non_checkout(day))
        
    filled_data = []
    weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday']
    date = ['02', '03', '04', '05', '06']
    for i, day in enumerate(days_data):
        #print(i)
        #print(day)
        temporary = add_checkout(day, customers_to_checkout[i], date[i], weekdays[i])
        filled_data.append(temporary)
        
    total = pd.concat(filled_data)
    #per_minute(total)
    print(total)

    
    #print(total.head(35))
    #print(total.head(35))
    print(len(total))
    #total.reset_index()
    
    #total.to_csv('./data/total.csv', sep=',')

*** S T A R T ***
data was loaded.

 *** N E W   C O L U M N S ***
new columns were added.

 *** A D D   N O   C H E C K O U T ***
a list of non checkouts was built
a list of non checkouts was built
a list of non checkouts was built
a list of non checkouts was built
a list of non checkouts was built
the checkout was added to the data-frame/customers for monday.
the checkout was added to the data-frame/customers for tuesday.
the checkout was added to the data-frame/customers for wednesday.
the checkout was added to the data-frame/customers for thursday.
the checkout was added to the data-frame/customers for friday.
                     customer_no  location     day    day_id
2019-09-02 07:03:00            1     dairy  monday     1-mon
2019-09-02 07:03:00            2     dairy  monday     2-mon
2019-09-02 07:04:00            3     dairy  monday     3-mon
2019-09-02 07:04:00            4     dairy  monday     4-mon
2019-09-02 07:04:00            5    spices  monday     5-mon
...         

In [33]:
total = total.groupby(['day_id']).resample('1Min').ffill()
total

customer_no  location        day   day_id
day_id                                                                
1-fri   2019-09-06 07:00:00            1     dairy     friday    1-fri
        2019-09-06 07:01:00            1     dairy     friday    1-fri
        2019-09-06 07:02:00            1     dairy     friday    1-fri
        2019-09-06 07:03:00            1     dairy     friday    1-fri
        2019-09-06 07:04:00            1    spices     friday    1-fri
...                                  ...       ...        ...      ...
999-tue 2019-09-03 18:00:00          999     dairy    tuesday  999-tue
        2019-09-03 18:01:00          999     dairy    tuesday  999-tue
        2019-09-03 18:02:00          999  checkout    tuesday  999-tue
999-wed 2019-09-04 16:53:00          999     fruit  wednesday  999-wed
        2019-09-04 16:54:00          999  checkout  wednesday  999-wed

[54192 rows x 4 columns]

In [34]:
def generate_prob_matrix(df):
    """This function takes a dataframe with customer movements over time inside the store. It generates a probability matrix of size N x N. where N is the total number of locations in the store. The matrix contains the probabilities that a customer will go to the next location based on the current one within the time defined by the timestep in the dataframe. It is meant to be used further in a Markov chain Monte Carlo simulation.

    INPUT ARGUMENTS:
        -df -> pandas DataFrame with a timedate index, customer_ID and location at every timestep.

    OUTPUT:
        -prob_matrix -> matrix where each element is the probability to transition from one state to another in a Markov chain simulation. 
    """
    
    # create empty series that will be populated with the current and previous locations for each customer
    loc_now = pd.Series(dtype=object)
    loc_before = pd.Series(dtype=object)
    
    # iterate through all customer IDs
    for customer_id in df.day_id.unique():
        
        # find the current and previous locations for just the current customer
        loc_per_customer = df[df['day_id'] ==  customer_id]['location']
        loc_per_customer_before = loc_per_customer.shift(1)
        
        # shifting data will create NaN values for previous locations; they will be replaced with 'entrance'
        loc_per_customer_before.fillna('entrance',
                               inplace=True)
        
        # add the current and previous locations to the series
        loc_now = pd.concat([loc_now,
                            loc_per_customer])
        loc_before = pd.concat([loc_before,
                                loc_per_customer_before])    
    
    # generate transition probability matrix
    global prob_matrix
    prob_matrix = pd.crosstab(loc_before, 
                            loc_now,
                            normalize='index')
    #print(prob_matrix)
    return prob_matrix, loc_before, loc_now

In [48]:
x = generate_prob_matrix(total)

In [49]:
transition_probs = x[0]

In [93]:
class Customer():
    '''
    Customer class for movement, activity and something
    '''
    
    def __init__(self, name , location, transition_probs):
        self.name = name    # is it possible to display the day_id here?
        self.location = location
        self.transition_probs = transition_probs

    def is_active(self):
        '''
        returns True if the customer has not reached the checkout yet.
        '''
        print('i´m moving.')
        
        if self.location != 'checkout':
             return True
        if self.location == 'checkout':
            return False

    def next_state(self):
        '''
        Propagates the customer to the next state
        using a weighted random choice from the transition probabilities
        conditional on the current state.
        Returns nothing.
        '''
        if self.is_active():
            a = list(self.transition_probs.loc[self.location].keys())
            b = self.transition_probs.loc[self.location].values[:]
            self.location = random.choices(a,b)[0]
            print(f'i just want to move to {self.location}.')
        
                
    def simulate_movement(self, num_steps):
        """
        Simulates the movement of the customer for the given number of steps.
        Returns a list of customer states after each step.
        """
        customer_states = [str(self)]
        #print(customer_states)
        for _ in range(num_steps):
            if self.location != 'checkout':
                self.next_state()
                customer_states.append(str(self))
        return customer_states
    
    def __repr__(self):
        return f'<Customer {self.name} in {self.location}>'

In [94]:
cust1 = Customer('Jake', 'entrance', transition_probs)
cust2 = Customer('Margaret', 'checkout', transition_probs)

In [95]:
custumoer_states_1 = cust1.simulate_movement(15)

i´m moving.
i just want to move to dairy.
i´m moving.
i just want to move to dairy.
i´m moving.
i just want to move to dairy.
i´m moving.
i just want to move to dairy.
i´m moving.
i just want to move to fruit.
i´m moving.
i just want to move to checkout.


In [85]:
print(cust1.name, cust1.location)
print(cust2)

Jake entrance
<Customer Margaret in checkout>


In [86]:
cust1.next_state()

i´m moving.
i just want to move.
